<a href="https://colab.research.google.com/github/Ariamb/treinamento-h2ia/blob/main/ID3_mushroom_tree_hub2ia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import io
import math



In [ ]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/mushroom/agaricus-lepiota.data


dados = pd.read_csv("agaricus-lepiota.data", header=None)
#data = pd.read_csv('/kaggle/input/playtennis/PlayTennis.csv')

dados

--2023-03-24 18:15:31--  https://archive.ics.uci.edu/ml/machine-learning-databases/mushroom/agaricus-lepiota.data
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 373704 (365K) [application/x-httpd-php]
Saving to: ‘agaricus-lepiota.data.5’

agaricus-lepiota.da 100%[===================>] 364.95K  1.29MB/s    in 0.3s    

2023-03-24 18:15:31 (1.29 MB/s) - ‘agaricus-lepiota.data.5’ saved [373704/373704]



,0,1,2,3,4,5,6,7,8,9,...,13,14,15,16,17,18,19,20,21,22
0,p,x,s,n,t,p,f,c,n,k,...,s,w,w,p,w,o,p,k,s,u
1,e,x,s,y,t,a,f,c,b,k,...,s,w,w,p,w,o,p,n,n,g
2,e,b,s,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,n,m
3,p,x,y,w,t,p,f,c,n,n,...,s,w,w,p,w,o,p,k,s,u
4,e,x,s,g,f,n,f,w,b,k,...,s,w,w,p,w,o,e,n,a,g
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8119,e,k,s,n,f,n,a,c,b,y,...,s,o,o,p,o,o,p,b,c,l
8120,e,x,s,n,f,n,a,c,b,y,...,s,o,o,p,n,o,p,b,v,l
8121,e,f,s,n,f,n,a,c,b,n,...,s,o,o,p,o,o,p,b,c,l
8122,p,k,y,n,f,y,f,c,n,b,...,k,w,w,p,w,o,e,w,v,l


In [ ]:
# Usando as funções de entropia do professor Ricardo

# Cálculo da entropia para o conjunto de dados S
def entropia(S):
  p1 = len(S[S[0]=='p']) / len(S)
  p0 = 1 - p1
  if p0==0 or p1==0:
    return 0
  else:
    return -p0*np.log2(p0) - p1*np.log2(p1)

# Cálculo do ganho de informação para o atributo A e conjunto de dados S
def ganho(S,A):
  valores = S[A].unique()
  soma = 0
  for valor in valores:
    Sv = S[S[A]==valor]
    pv = len(Sv) / len(S)
    Ev = entropia(Sv)
    soma = soma + pv*Ev
  return entropia(S) - soma

In [ ]:
class Nodo:
    def __init__(self):
        self.atributo = None
        self.arestas_e_classes = [] #vetor de tuplas: (valor da coluna, ref a aresta)
      
    def percorre(self): #só para visualização da arvore em pré-ordem
      print('nodo : ', self.atributo)
      for x in self.arestas_e_classes:
        x[1].percorre()

    def decide(self, atributos): 
        x = atributos[self.atributo]
        for a in self.arestas_e_classes:
          if a[0] == x:
            return a[1].decide(atributos)
    
class Aresta:
      def __init__(self):
        self.valor_coluna = None
        self.filho = None
      def percorre(self):
        print('aresta: ', self.valor_coluna)
        self.filho.percorre()
      def decide(self, atributos):
        return self.filho.decide(atributos)

class Folha:
    def __init__(self):
        self.resultado = None #'p' ou 'e', ou 'yes' ou 'no' ou 1 ou 0
    def percorre(self):
        print('folha :', self.resultado)
    def decide(self, atributo):
      return self.resultado

In [ ]:
# Para cada atributo, calculamos o ganho de informação
from collections import Counter

atributos = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22]
#VETOR FEITO A MÃO DE PROPÓSITO
#EU SEI FAZER ELE USANDO RANGE
#list(range(1, 23))


def cria_arvore(dados, depth, max_depth):
    
  S = dados
  tuples = []

  for A in atributos: #encontra qual atributo tem o melhor ganho dentre todos
    thistuple = (A, ganho(S, A))
    tuples.append(thistuple)
  bigger = tuples[0]
  for x in tuples:
    if x[1] >= bigger[1]:
      bigger = x

  if bigger[1] == 0 or depth == max_depth: #poda da árvore por profundidade
  #caso ganho for zero, todo mundo é do mesma classe e adiciona uma folha
    c = Counter(S[0])
    f = Folha()
    f.resultado = c.most_common()[0][0]
    return f


  pai = Nodo()
  pai.atributo = bigger[0]
  possible_child_nodes = (S[pai.atributo].unique())
  for x in possible_child_nodes:
    aresta = Aresta()
    aresta.valor_coluna = x
    pai.arestas_e_classes.append((x, aresta))    
    SL = S[S[pai.atributo]==aresta.valor_coluna]
    aresta.filho = cria_arvore(SL, depth+1, max_depth)
  return pai


In [ ]:
from sklearn.model_selection import train_test_split

treino, teste = train_test_split(dados, test_size=0.2)
print(treino)
print(teste)

     0  1  2  3  4  5  6  7  8  9   ... 13 14 15 16 17 18 19 20 21 22
5382  e  f  y  b  t  n  f  c  b  e  ...  s  e  w  p  w  t  e  w  c  w
3887  p  x  s  w  f  c  f  c  n  n  ...  s  w  w  p  w  o  p  n  s  d
4059  e  x  f  e  t  n  f  c  b  n  ...  s  w  p  p  w  o  p  n  v  d
2464  e  f  f  n  t  n  f  c  b  u  ...  s  p  w  p  w  o  p  n  y  d
6707  p  k  y  n  f  y  f  c  n  b  ...  k  p  w  p  w  o  e  w  v  l
...  .. .. .. .. .. .. .. .. .. ..  ... .. .. .. .. .. .. .. .. .. ..
6089  p  f  s  n  f  s  f  c  n  b  ...  k  p  w  p  w  o  e  w  v  d
4408  p  f  y  g  f  f  f  c  b  g  ...  k  n  b  p  w  o  l  h  y  g
7325  e  f  s  n  f  n  a  c  b  o  ...  s  o  o  p  n  o  p  y  c  l
3457  p  x  f  w  f  c  f  w  n  g  ...  s  w  w  p  w  o  p  k  s  d
2924  e  x  y  g  t  n  f  c  b  n  ...  s  p  w  p  w  o  p  k  y  d

[6499 rows x 23 columns]
     0  1  2  3  4  5  6  7  8  9   ... 13 14 15 16 17 18 19 20 21 22
4867  p  x  f  y  f  f  f  c  b  h  ...  k  p  b  p  w  o  l  h 

In [ ]:
def calcula_stats(tree, teste):
  #                    real = class
  TP = 0 #true positive - p = p
  TN = 0 #true negative - e = e
  FP = 0 #false positive  e = p
  FN = 0 #false negative  p = e
  for row in teste.iterrows():
    atb = row
    pred = tree.decide(atb[1])
    real = atb[1][0]
    if (pred == 'p') & (pred == real):
      TP += 1
    if (pred == 'e') & (pred == real):
      TN += 1
    if (pred == 'e') & (pred != real):
      FP += 1
    if (pred == 'p') & (pred != real):
      FN += 1
      
  print('acertos: ', TP + TN) # só pra minha visualização
  print('erros: ', FP + FN)


  ACC = (TP + TN) / (TP + FP + TN + FN)
  print('accuracy: ', ACC)
  PRE = (TP) / (TP + FP)
  print('precision: ', PRE)
  REC = TP / (TP + FN)
  print('recall: ', REC)
  SPE = TN / (TN + FP)
  print('specifity: ', SPE)

  F1 = 2 * (PRE * REC) / (PRE + REC)
  print("F1 score: ", F1)


  return 0

In [ ]:
print('PRUNNING AT DEPTH 3:')

tree = cria_arvore(treino, 0, 3)
calcula_stats(tree, teste)

print('\n')
print('PRUNNING AT DEPTH 4:')

tree = cria_arvore(treino, 0, 4)
calcula_stats(tree, teste)

PRUNNING AT DEPTH 3:
acertos:  1619
erros:  6
accuracy:  0.9963076923076923
precision:  0.9961240310077519
recall:  0.9961240310077519
specifity:  0.9964747356051704
F1 score:  0.9961240310077519


PRUNNING AT DEPTH 4:
acertos:  1625
erros:  0
accuracy:  1.0
precision:  1.0
recall:  1.0
specifity:  1.0
F1 score:  1.0


0